In [5]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

from sklearn.metrics.pairwise import cosine_similarity

credits_df = pd.read_csv("/content/credits.csv")
movies_df = pd.read_csv("/content/movies.csv")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

movies_df = movies_df.merge(credits_df, on = 'title')
movies_df.head()

movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

movies_df.dropna(inplace = True)

def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
      l.append(i['name'])
    return l

movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)

def convert3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
      if counter != 3:
        l.append(i['name'])
        counter += 1
      else:
        break
    return l


movies_df['cast'] = movies_df['cast'].apply(convert3)

def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
      if i['job'] == 'Director':
        l.append(i['name'])
        break
    return l

movies_df['crew'] = movies_df['crew'].apply(fetch_director)

movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())

movies_df['genres'] = movies_df['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']

new_df = movies_df[['movie_id', 'title', 'tags']].copy()

new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x:" ".join(x))

new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x:x.lower())

cv = CountVectorizer(max_features = 5000, stop_words = 'english')

cv.fit_transform(new_df['tags']).toarray().shape
vectors = cv.fit_transform(new_df['tags']).toarray()


len(cv.get_feature_names_out())

def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

new_df.loc[:, 'tags'] = new_df['tags'].apply(stem)

cosine_similarity(vectors).shape

similarity = cosine_similarity(vectors)

sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x:x[1])[1:6]

def recommend(movie):
  movie_index = new_df[new_df['title'] == movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:6]
  for i in movies_list:
    print(new_df.iloc[i[0]].title)


In [13]:
recommend("Rango")

Blazing Saddles
Alvin and the Chipmunks: The Road Chip
Over the Hedge
Return to Oz
A Million Ways to Die in the West
